In [22]:
# 필요 패키지 실행
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import requests
import bs4

### 인포스탁 크롤링 진행

In [28]:
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# 국내 주식 소분류(테마)명 추출
def scrape_text_elements(url):
    response = requests.get(url).content.decode('cp949')
    response = bs4.BeautifulSoup(response, 'html.parser')
    target = response.find('tbody')
    td_elements = target.find_all('td', class_='alL')
    texts = [td.find('a').text for td in td_elements]
    return texts

def go_back(driver):
    driver.back()

# 소분류(테마)별 종목명 추출
def extract_stock_names(driver, texts, theme_dic):
    names = []
    td_elements = driver.find_elements(By.CSS_SELECTOR, 'td.alL')
    for td in td_elements:
        a_tag = td.find_element(By.TAG_NAME, 'a')
        time.sleep(2)
        tm = a_tag.text
        if tm in texts:
            a_tag.click()
            time.sleep(1)
            contents_element = driver.find_element(By.ID, "contents")
            tbody_element = contents_element.find_element(By.TAG_NAME, "tbody")
            tr_elements = tbody_element.find_elements(By.TAG_NAME, "tr")
            for tr_element in tr_elements:
                style1_elements = tr_element.find_elements(By.CSS_SELECTOR, "span.style1")
                for style1_element in style1_elements:
                    a_elements = style1_element.find_elements(By.TAG_NAME, "a")
                    if a_elements:
                        for a_element in a_elements:
                            name = a_element.text
                            names.append(name)
            theme_nm = tm.replace('◎', '').strip()
            theme_dic[theme_nm] = names.copy()
            time.sleep(1)
            go_back(driver)
            time.sleep(1)
            names = []
            
#드라이버 실행
def scrape_stock_names(url):
    texts = scrape_text_elements(url)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)

    theme_dic = {}

    extract_stock_names(driver, texts, theme_dic)

    driver.quit()

    return theme_dic

# Main program
url = "http://m.infostock.co.kr/sector/sector.asp?mode=n"
theme_dic = scrape_stock_names(url)

theme = pd.DataFrame([(k, v) for k, values in theme_dic.items() for v in values], columns=['소분류', '기업'])

print(theme)


                     소분류         기업
0          2022 상반기 신규상장        오토앤
1          2022 상반기 신규상장  DB금융스팩10호
2          2022 상반기 신규상장     애드바이오텍
3          2022 상반기 신규상장       케이옥션
4          2022 상반기 신규상장   LG에너지솔루션
...                  ...        ...
5773  U-Healthcare(원격진료)       케어랩스
5774  U-Healthcare(원격진료)      이지케어텍
5775  U-Healthcare(원격진료)       네오펙트
5776  U-Healthcare(원격진료)      제이엘케이
5777  U-Healthcare(원격진료)    라이프시맨틱스

[5778 rows x 2 columns]


In [27]:
theme.to_csv('국내 주식 소분류.csv', index = False, encoding = 'cp949')